# 데이터 블록 만드는 법 (1)

> fastai에서는 데이터를 정의하는 방법으로 DataBlock API를 제안합니다. 각 인자가 의미하는 내용과, 실제 Siamese 공식 튜토리얼에 이 내용이 어떻게 적용되는지를 살펴봅니다.
- author: "Chansung Park"
- toc: true
- image: images/datablock/siamese-block.png
- comments: true
- categories: [datablock, siamese, fastai]
- permalink: /datablock-siamese/
- badges: false
- search_exclude: true

In [65]:
#hide
!pip install fastai
!pip install nbdev

In [66]:
#hide
from fastai.vision.all import *
import nbdev

In [67]:
#hide
path = untar_data(URLs.PETS)

# DataBlock API

In [59]:
nbdev.show_doc(DataBlock)

<h2 id="DataBlock" class="doc_header"><code>class</code> <code>DataBlock</code><a href="https://github.com/fastai/fastai/tree/master/fastai/data/block.py#L58" class="source_link" style="float:right">[source]</a></h2>

> <code>DataBlock</code>(**`blocks`**=*`None`*, **`dl_type`**=*`None`*, **`getters`**=*`None`*, **`n_inp`**=*`None`*, **`item_tfms`**=*`None`*, **`batch_tfms`**=*`None`*, **`get_items`**=*`None`*, **`splitter`**=*`None`*, **`get_y`**=*`None`*, **`get_x`**=*`None`*)

Generic container to quickly build `Datasets` and `DataLoaders`

**DataBlock**은 모델이 학습하는 데이터를 준비시키는 핵심 API 입니다. 중요한 사실은 **DataBlock**은 일종의 **템플릿** 이라는 것입니다. 실제로 데이터가 **주입되었을 때**, **'이런 이런 식으로 동작한다'** 를 정의하는 것이죠.

상기 **DataBlock** API의 원형을 말로 풀어서 설명하면 다음과 같습니다.

- **dls = siamese.dataloaders(files)**
  - 이 부분은 DataBlock을 만든 후 수행되는 코드입니다.
  - 입력된 것은 1이 수용할 데이터 목록입니다.


- **get_items**: 수용된 데이터를 논리적인 집합 목록으로 만듭니다.
  - 이 함수는 무엇을 반환해도 상관이 없습니다. 단, 반환된 값으로 입력될 데이터와 레이블의 추출이 가능해야 합니다.
  - 이 단계에서 다뤄지는 **데이터**란, 아무런 변환 처리가 적용되지 않은 **raw** 입니다.
    - *예시 1) 이미지 파일 경로의 목록 (경로에서 입력으로 사용될 파일이름과, 상위폴더로 결정 가능한 레이블이 모두 포함되어 있다면 OK)*
    - *예시 2) (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플들의 목록*


- **get_x**: 무엇을 입력으로 삼을지 결정합니다.
  - **get_items**이 반환한 목록을 하나씩 접근해서 처리합니다.
  - 이 단계에서도 다뤄지는 데이터는 raw 입니다.
    - *예시 1) 이미지 파일 경로를 그대로 반환 (bypass)*
    - *예시 2) (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플에서 1과 2를 추출해서 반환*


- **get_y**: 무엇을 레이블로 삼을지 결정합니다. 
  - **get_items**이 반환한 목록을 하나씩 접근해서 처리합니다.
  - 이 단계에서도 다뤄지는 데이터는 raw 입니다.
    - *예시 1) 이미지 파일 경로에서 상위 폴더명을 추출해서 반환*
    - *예시 2) (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플에서 3을 추출해서 반환*


- **blocks**: raw 형식의 데이터를 모델에 입력 가능한 형식으로 바꿀 규칙을 결정합니다.
  - 두 개 이상을 지정할 수 있습니다. **get_x** 및 **get_y** 의 내용을 모두 수용할 수 있어야 합니다. 
  - 입력과 출력을 구분하기 위한 목적으로 **n_inp** 라는 인자를 건드릴 수 있습니다.
    - *예시 1) 이미지 파일 경로에서, 이미지를 불러오고 tensor로 변환하는 ImageBlock. 상위 폴더명을 원-핫 인코딩된 tensor로 변환하는 CategoryBlock*
    - *예시 2-1) 이미지 파일 두 개를 수용하기 위한, 두 개의 ImageBlock. 해당 레이블을 원-핫 인코딩 tensor로 변환하는 CategoryBlock*
    - *예시 2-2) 이미지 파일 두 개를 한번에 수용 가능한 TupleBlock. 해당 레이블을 원-핫 인코딩 tensor로 변환하는 CategoryBlock*
     

- **item_tfms**: 각 데이터 하나에 대한 변형을 하고 싶다면, 그 변형 규칙을 지정합니다.


- **batch_tfms**: 배치 단위의 데이터에 대한 변형을 하고 싶다면, 그 변형 규칙을 지정합니다.
  - 배치 단위의 데이터 변형은 배치 단위로 GPU(가용하다면) 에서 수행됩니다.
   
- **splitter**: 학습/검증 데이터셋을 구분하는 방법을 결정합니다.
  - 두 개(학습/검증)를 포함한 튜플 형식을 반환합니다.
  - 각각 리스트이며, 리스트에는 인덱스 목록이 들어 있습니다. **get_items** 에서 누구 인덱스를 학습용으로, 누구 인덱스를 검증용으로 둘 것인지를 나열한 것입니다.

In [15]:
siamese = DataBlock(   
    get_items=get_tuples,                       # 모든 데이터를 불러 들이는 함수를 지정합니다.
        get_x=get_x,                            # 불러와진 데이터에 대해서, 입력을 결정하는 함수를 지정합니다.
        get_y=get_y,                            # 불러와진 데이터에 대해서, 출력을 결정하는 함수를 지정합니다.
       blocks=(ImageTupleBlock, CategoryBlock), # tuple 형식으로, 두 개 이상도 가능합니다.
    item_tfms=Resize(224),                      # 아이템 단위의 변환
   batch_tfms=[Normalize.from_stats(*imagenet_stats)],   # 배치 단위의 변환
     splitter=splitter                         # 학습/검증 데이터셋을 분리하는 함수를 지정합니다.        
)

# Siamese 예제

앞선 설명을 Siamese 데이터 블록에 대입해서 설명해 보자면

- **siamese.dataloaders(files)**
  - files는 단순히 Path 객체로 표현된 이미지 파일 경로의 목록 입니다.


- **get_items**: get_tuples
  - get_tuples 함수는 입력된 이미지 파일 경로 목록들로부터, 튜플 (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 목록을 만들어 반환합니다. 
    - 우선 각 파일을 순차적으로 접근합니다. 접근될 때마다 무작위로 다른 파일을 하나 접근합니다.
    - 그리고 두 파일의 카테고리를 추출합니다. 두 파일의 카테고리가 일치한다면 True, 아니라면 False를 레이블로서 정합니다.
  
  
- **get_x**: get_x
  - get_x 함수는 **get_tuples** 가 반환한 목록을 하나씩 접근합니다.
    - 즉, (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플을 하나씩 처리합니다.
    - 해당 튜플 중 처음 두 개만이 입력 데이터이므로 튜플[:2] 를 반환합니다.
  
  
- **get_y**: get_y
  - get_y 함수는 **get_tuples** 가 반환한 목록을 하나씩 접근합니다.
    - 즉, (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플을 하나씩 처리합니다.
    - 해당 튜플 중 마지막 하나만이 출력 데이터이므로 튜플[2]를 반환합니다.


- **blocks**: (ImageTupleBlock, CategoryBlock)
  - ImageTupleBlock은 내부적으로 두 개의 이미지를 입력받고, 이들을 PILImage 형식으로 변환 후 TensorImage 형식으로 변환합니다.
    - ImageTupleBlock은 TransformBlock을 반환하는데, TransformBlock에는 type_tfms 및 batch_tfms 인자가 있습니다. type_tfms는 DataBlock의 item_tfms와 동일한 것으로 실제로는 머지되어 처리됩니다. 마찬가지로 TransformBlock에서 지정된 batch_tfms도 DataBlock의 batch_tfms와 머지되어 처리됩니다.
      - type_tfms에서 PILImage로의 변환 작업이 지정되고, batch_tfms에서 TensorImage로의 변환 작업이 지정되었습니다.
  - CategoryBlock은 주어진 레이블을 원-핫 인코딩된 tensor로 변환합니다.


- **item_tfms**: Resize(224)
   - 이미지 크기 조절은 Resize 라는 Transform 형 객체를 활용합니다.
   - item_tfms에 나열되는 Transform 류는 준비된 데이터 튜플(이미지 파일 경로 1, 이미지 파일 경로 2, 레이블)에 모두 적용됩니다.
     - 하지만, 내부적으로 자신이 적용 가능한 녀석이 아니면 그대로 종료됩니다. 즉, 이미지 파일 경로 1, 이미지 파일 경로 2에 대한 변환만이 수행됩니다.



- **batch_tfms**: [Normalize.from_stats(*imagenet_stats)]
   - **item_tfms** 때와 마찬가지로 적용될 대상을 스스로 알아챕니다
   - **Normalize.from_stats(*imagenet_stats)**는 이미지넷 학습에 사용된 데이터의 평균, 표준편차 값으로 전이학습 되는 데이터의 정규화를 수행한다는 의미를 가집니다. 그렇게 해서 전이학습 대상 모델이 학습한 데이터와 유사한 값들로 변환합니다.

## get_items: get_tuples

**(이미지 파일 경로 1, 이미지 파일 경로 2, 레이블)** 튜플 목록을 구성하는데 **get_tuples** 함수가 지정되었습니다. **get_tuples** 함수는 다른 함수와 변수가 활용되어 만들어져 있습니다. 각각을 살펴보도록 하겠습니다.

1. **draw_other(filename)**
  1. 주어진 파일이름으로부터 카테고리를 추출합니다. (**category_extraction_func**)
  1. 주어진 파일이름이 속한 데이터셋을 알아냅니다. (학습? 검증?). (**get_split**)
  1. 주어진 파일과 비교될 다른 이미지를 선택합니다. 
    1. 카테고리가 같은지, 다른지에 대한 무작위성 확률을 생성합니다. (**is_same**)
  1. 무작위로 만들어진 확률에 따라, 비교될 이미지의 카테고리를 무작위로 선택합니다.
  1. 선택된 카테고리내에 속한 이미지를 무작위로 선택합니다.
  1. **(선택된 이미지 파일 경로, is_same)**를 반환합니다.
  
  
2. **get_tuples(filenames)**
  1. 모든 파일이름을 순차적으로 접근하여, **[filename, *draw_other(filename)]**를 만듭니다.
    1. **draw_other(filename)**는 **(선택된 이미지 파일 경로, is_same)**를 반환하였습니다.
    1. *draw_other(filename)**은 반환된 두 값을 list인 것처럼 만들고, 이를 **[filename]** 에 편승시킵니다.
    1. 따라서, 논리적으로 볼 때 **[filename, 선택된 이미지 파일 경로, is_same]**, 세 값으로 구성된 list가 만들어집니다.
  1. 만들어진 **[filename, *draw_other(filename)]** 들을 list 목록으로 만듭니다. 
  
  
이제 각각을 살펴보겠습니다.

### category_extraction_func

이미지 파일의 경로가 주어졌을 때, 경로에서 레이블을 추출하는 함수입니다. 
- 정규표현식을 사용하고 있으며, 아래 코드의 **r'^(.*)_\d+.jpg$'** 부분이 정규표현식에 해당합니다. 
- 정규표현식에 맞는 문자열을 발견했다면, 괄호 **( )** 안의 것을 추출하여 반환합니다.

In [68]:
def category_extraction_func(filename):
    return re.match(r'^(.*)_\d+.jpg$', filename.name).groups()[0] # () 안의 것을 추출하여 반환

아래의 테스트 코드는 "/Users/chansung/.fastai/data/oxford-iiit-pet/images/**Egyptian_Mau**_167.jpg" 라는 파일 경로에서, **Egyptian_Mau**가 추출되는 것을 보여줍니다.

In [69]:
print(files[0])
print(category_extraction_func(files[0]))

/Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_167.jpg
Egyptian_Mau


### 카테고리 - 파일 경로 매핑 테이블

#### 전체 파일 로딩 및 카테고리 테이블 생성

앞서 간단히 설명된 **draw_other** 및 **get_tuples** 함수에서는 주어진 이미지와 다른 카테고리의 이미지를 선택하는 로직이 있었습니다. 다른 이미지를 선택하기 위해서는 카테고리별로 이미지 경로의 목록이 정리되어 있어야 할 것입니다. 다음의 세 코드 블록은 이러한 매핑 테이블을 구성하는 방법을 보여줍니다.

1. **우선 모든 이미지 파일 목록을 찾습니다.**
  1. **get_image_files**는 입력된 경로로부터, 재귀적으로 모든 하위 경로를 탐색하여 이미지 파일들**만**을 추출하여 목록으로서 반환합니다.
  1. **get_image_files**는 fastai에서 제공하는 유틸리티 함수입니다.

In [70]:
nbdev.show_doc(get_image_files)

<h4 id="get_image_files" class="doc_header"><code>get_image_files</code><a href="https://github.com/fastai/fastai/tree/master/fastai/data/transforms.py#L55" class="source_link" style="float:right">[source]</a></h4>

> <code>get_image_files</code>(**`path`**, **`recurse`**=*`True`*, **`folders`**=*`None`*)

Get image files in `path` recursively, only in `folders`, if specified.

In [71]:
files      = get_image_files(path/"images")  # 지정 디렉토리 아래에 존재하는 모든 이미지 파일 목록 긁어오기

아래의 결과를 보면 알 수 있듯이 모든 이미지 파일의 경로 목록이 만들어졌습니다. (학습/검증용 데이터를 가리지 않고, 모두 뽑아냅니다. 이후 splitter 부분에서 누구를 학습, 누구를 검증 데이터셋에 넣을지를 정합니다)

In [72]:
files

(#7390) [Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_167.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/pug_52.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/basset_hound_112.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/Siamese_193.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/shiba_inu_122.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/Siamese_53.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/Birman_167.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/leonberger_6.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/Siamese_47.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/shiba_inu_136.jpg')...]

2. **추출된 이미지 파일 경로를 순차적으로 접근하여, category_extraction_func 함수로 카테고리를 추출합니다.** 
  1. 그리고, 추출된 카테고리를 set에 넣어줍니다. 즉, 중복성을 제거하여 고유한 카테고리 종류가 몇 가지 있는지를 파악합니다. 
  1. 이렇게 만들어진 set은 나중에 카테고리 - 파일 경로 매핑 테이블 구성시 **카테고리** 키값을 다룰 때 활용됩니다.

In [73]:
categories = list(set(files.map(category_extraction_func)))    # 카테고리 목록 만들기

아래 코드의 결과를 통해서 총 37종의 카테고리가 있음을 알 수 있습니다.

In [74]:
print(len(categories))
print(categories)

37
['basset_hound', 'Bombay', 'boxer', 'scottish_terrier', 'wheaten_terrier', 'pomeranian', 'great_pyrenees', 'japanese_chin', 'keeshond', 'havanese', 'Sphynx', 'english_setter', 'shiba_inu', 'Siamese', 'newfoundland', 'Egyptian_Mau', 'yorkshire_terrier', 'staffordshire_bull_terrier', 'beagle', 'Bengal', 'Ragdoll', 'chihuahua', 'english_cocker_spaniel', 'Russian_Blue', 'miniature_pinscher', 'saint_bernard', 'Abyssinian', 'Maine_Coon', 'British_Shorthair', 'leonberger', 'samoyed', 'german_shorthaired', 'american_bulldog', 'american_pit_bull_terrier', 'Persian', 'Birman', 'pug']


#### 학습과 검증용 데이터셋의 분리

갑자기 학습/검증용 데이터셋 분리 내용이 나옵니다. 두 이미지의 선택은 각 데이터셋에서 이루어 져야 하기 때문에 이 과정을 먼저 다룹니다. 학습용 데이터에 검증용 데이터가 섞이면 안되겠죠? 즉, (이미지 파일 경로 1, 이미지 파일 경로 2)의 두 이미지는 같은 데이터셋 내에서 골라지는 것입니다.

1. **우선 전체 데이터를 학습/검증용으로 쪼갭니다.**
  1. RandomSplitter는 list를 입력받고, 이를 정해진 비율로 쪼갭니다.
  1. 쪼개어진 결과는 [ [ ] , [ ] ] 형식이 되고, 각 list에 저장되는 것은 실제 데이터가 아니라 인덱스 번호 입니다.


In [75]:
nbdev.show_doc(RandomSplitter)

<h4 id="RandomSplitter" class="doc_header"><code>RandomSplitter</code><a href="https://github.com/fastai/fastai/tree/master/fastai/data/transforms.py#L85" class="source_link" style="float:right">[source]</a></h4>

> <code>RandomSplitter</code>(**`valid_pct`**=*`0.2`*, **`seed`**=*`None`*)

Create function that splits `items` between train/val with `valid_pct` randomly.

In [76]:
splits       = RandomSplitter()(files)                         # 디폴트 valid_pct 값은 0.2. 반환되는 값은 인덱스 목록의 튜플

아래 결과처럼, 두 개의 list가 만들어졌습니다. 디폴트로 RnadomSplitter는 80:20 비율로 두 데이터셋을 나눕니다.

In [77]:
splits

((#5912) [6568,2440,1610,6972,4603,4976,1898,645,1781,3439...],
 (#1478) [3082,5775,3635,706,3729,5408,7362,7213,96,443...])

2. **그 다음은 RandomSplitter가 반환한 인덱스 list로부터 실제 데이터를 가져옵니다.**
  1. 단순히 인덱스 번호를 참조해서, files로부터 매핑된 파일 목록을 추출하는 과정입니다.
  1. 그 다음, 파일의 중복성을 제거하기 위해서 set 으로 만들어줍니다. (반드시 필요한 과정은 아닙니다)

In [78]:
splits_files = [files[splits[i]] for i in range(2)]            # splits 정보를 기반으로, 실제 이미지 파일 경로 목록을 구성
splits_sets  = mapped(set, splits_files)                       # 중복 파일이 없는지 확인하기 위해서 set으로 만듬

아래 결과처럼, 두 개의 list가 만들어졌습니다. 단, 이번에는 실제 이미지 파일 경로 목록이 들어갔습니다.

In [86]:
splits_files

[(#5912) [Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/chihuahua_44.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/staffordshire_bull_terrier_170.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/american_pit_bull_terrier_200.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/english_setter_154.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/newfoundland_173.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/Siamese_101.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/Persian_256.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/english_cocker_spaniel_194.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/english_setter_160.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/scottish_terrier_57.jpg')...],
 (#1478) [Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/samoyed_113.jpg'),Path('/Users/chansung/.fastai/data/oxford-iiit-pet/images/A

#### 학습과 검증용 데이터셋의 분리

In [90]:
# 동일한 카테고리에 속하는 이미지 파일끼리 묶어서, 딕셔너리 형태로 관리
splbl2files  = [{c: [f for f in s if category_extraction_func(f) == c] for c in categories} for s in splits_sets]

def get_split(f):
    for i,s in enumerate(splits_sets):
        if f in s: return i
    raise ValueError(f'File {f} is not presented in any split.')

def draw_other(filename):
    given_category = category_extraction_func(filename)
    split    = get_split(filename)

    is_same  = random.random() < 0.5    
    
    if not is_same: 
        other_category = random.choice(L(category for category in categories if category != given_category)) 

    return random.choice(splbl2files[split][other_category]), is_same

def get_tuples(filenames): 
    return [[filename, *draw_other(filename)] for filename in filenames]

In [48]:


def splitter(items): 
    def get_split_files(i): 
        return [j for j,(f1,f2,same) in enumerate(items) if get_split(f1)==i]
    
    return get_split_files(0),get_split_files(1)

In [49]:
# 동일한 카테고리에 속하는 이미지 파일끼리 묶어서, 딕셔너리 형태로 관리
splbl2files  = [{c: [f for f in s if category_extraction_func(f) == c] for c in categories} for s in splits_sets]

In [41]:
class ImageTuple(fastuple):
    @classmethod
    def create(cls, fns): return cls(tuple(PILImage.create(f) for f in fns))
    
    def show(self, ctx=None, **kwargs): 
        t1,t2 = self
        if not isinstance(t1, Tensor) or not isinstance(t2, Tensor) or t1.shape != t2.shape: return ctx
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        return show_image(torch.cat([t1,line,t2], dim=2), ctx=ctx, **kwargs)

def ImageTupleBlock():
    return TransformBlock(type_tfms=ImageTuple.create, batch_tfms=IntToFloatTensor)